In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from helpers import *

import warnings

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
plt.rcdefaults()

In [27]:
DATA_FOLDER = "../Data/"

movies_imdb = pd.read_csv(DATA_FOLDER + 'movies_imdb.csv',
                          converters={'languages': parse_list, 'countries': parse_list, 'genresCmu': parse_list,
                                      'genresImdb': parse_list})

# Methodologie

1. Prendre genre et creer un long str
2. couper aux delimiter qui sont pas des lettres et avoir une liste de touts les mots
3. unique tous les mots et en miniscules
4. voir comment les mots romantic et romance par exemple peut-être vus comme equivalent 


1.

In [28]:
genres = movies_imdb.dropna(subset="genresCmu").explode(column="genresCmu").genresCmu.values
genres_str = " ".join(str(g) for g in genres)

2.

In [29]:
split_characters = set()
for m in genres_str:
    for l in m:
        if not l.isalpha():
            split_characters.add(l)
            
split_characters

{' ', '&', "'", '-', '/', '5', '9', '[', ']'}

In [30]:
string = genres_str
for delimiter in split_characters:
    string = " ".join(string.split(delimiter))
    
words = string.split(" ")

3.

In [34]:
import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

words = list(map(lambda x: x.lower(), words)) #tout en minuscule
words = list(map(lambda x:lemmatizer.lemmatize(x), words)) #enlever les s
ags = nltk.pos_tag(words) # dire si c'est un verbe adjectif etc
words_, tags= np.array(list(map(list, zip(*ags))))

In [35]:
mask = (tags == np.array(["NN","NNS","NNP","NNPS","JJ"]).reshape(-1,1)).any(axis=0) # grader que les nouns et les adjs
words_len = np.array(list(map(lambda x : len(x),words_))) #taille des mots
mask = mask & (words_len>2) # garder que les mots de plus de 2 lettres 

In [36]:
counts = pd.Series(words_[mask]).value_counts() # compter combien de fois chaque mot apparait
pd.Series(counts[counts>1000]).index # les mots qui apparaissent plus de 1000 fois

Index(['film', 'drama', 'comedy', 'action', 'thriller', 'fiction', 'romance',
       'adventure', 'black', 'crime', 'white', 'short', 'cinema', 'world',
       'movie', 'family', 'indie', 'romantic', 'horror', 'documentary',
       'silent', 'musical', 'animation', 'fantasy', 'mystery', 'science',
       'war', 'western', 'japanese', 'period', 'child', 'political',
       'adaptation', 'chinese', 'art', 'biography', 'piece', 'psychological',
       'sport', 'biographical', 'bollywood', 'music', 'television',
       'historical', 'gay'],
      dtype='object')